In [7]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
odds_filter=1.7
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")
data=pd.read_sql_query("Select distinct * from results_clay_1 where date > '2024-03-27'",con=devengine)
data["Fav_Odds"] = data.Fav_Odds.astype(float)
data["Dog_Odds"] = data.Dog_Odds.astype(float)
data['Fav_Win'] = data['Winner']==data['Fav']
data=data[data['Fav_Odds']>odds_filter]

In [8]:
test_data = data.copy()
test_data1 = data.copy()
test_data['Winner_Dog'] = test_data['Dog'] == test_data['Winner']
test_data['Winner_EloDog'] = test_data['Elo_Fav'] != test_data['Winner']
column_order = ['Winner_Dog', 'Winner_EloDog'] + [col for col in test_data if col not in ['Winner_Dog', 'Winner_EloDog']]
test_data = test_data[column_order]
test_data=test_data.drop(columns=['Winner_Odds'])
test_data=test_data[[ 'fav_percent', 'dog_percent','fav_rank', 'dog_rank', 'Elo_Fav_Elo', 'Elo_Dog_Elo', 'Fav_Top100', 'Dog_Top100','Winner','Fav','Dog','Fav_Odds','Dog_Odds','Elo_Fav']]

# Create a single set of random indices for all columns
random_order = np.random.permutation(test_data.index)

# Create new columns with randomized order for specific columns
test_data['Player_1'] = test_data.apply(lambda row: np.random.choice([row['Fav'], row['Dog']]), axis=1)
test_data['Player_2'] = test_data.apply(lambda x: x['Fav'] if x['Player_1']==x['Dog'] else x['Dog'],axis=1)
test_data['Player_1_Odds'] = test_data.apply(lambda x: x['Fav_Odds'] if x['Player_1']!=x['Dog'] else x['Dog_Odds'],axis=1)
test_data['Player_2_Odds'] = test_data.apply(lambda x: x['Fav_Odds'] if x['Player_1']==x['Dog'] else x['Dog_Odds'],axis=1)
test_data['Player_1_Rank'] = test_data.apply(lambda x: x['fav_rank'] if x['Player_1']!=x['Dog'] else x['dog_rank'],axis=1)
test_data['Player_2_Rank'] = test_data.apply(lambda x: x['fav_rank'] if x['Player_1']==x['Dog'] else x['dog_rank'],axis=1)
test_data['Player_1_Top100'] = test_data.apply(lambda x: x['Fav_Top100'] if x['Player_1']!=x['Dog'] else x['Dog_Top100'],axis=1)
test_data['Player_2_Top100'] = test_data.apply(lambda x: x['Fav_Top100'] if x['Player_1']==x['Dog'] else x['Dog_Top100'],axis=1)
test_data['Player_1_percent'] = test_data.apply(lambda x: x['fav_percent'] if x['Player_1']!=x['Dog'] else x['dog_percent'],axis=1)
test_data['Player_2_percent'] = test_data.apply(lambda x: x['fav_percent'] if x['Player_1']==x['Dog'] else x['dog_percent'],axis=1)
test_data['FavEloFav']=test_data['Fav']==test_data['Elo_Fav']
test_data['Fav_Elo']=test_data.apply(lambda x: x['Elo_Fav_Elo'] if (x['FavEloFav']==True) else x['Elo_Dog_Elo'],axis=1)
test_data['Dog_Elo']=test_data.apply(lambda x: x['Elo_Fav_Elo'] if (x['FavEloFav']==False) else x['Elo_Dog_Elo'],axis=1)

test_data['Player_1_Elo'] = test_data.apply(lambda x: x['Fav_Elo'] if x['Player_1']!=x['Dog'] else x['Dog_Elo'],axis=1)
test_data['Player_2_Elo'] = test_data.apply(lambda x: x['Fav_Elo'] if x['Player_1']==x['Dog'] else x['Dog_Elo'],axis=1)
test_data['Player_1_Win'] = test_data['Player_1']==test_data['Winner']
#test_data['Player_2_Elo'] = test_data.apply(lambda x: x['Elo_Dog_Elo'] if ((x['Player_1']==x['Dog'])&(x['FavEloFav2']==True)) else x['Elo_Fav_Elo'],axis=1)
# Filter columns with the word 'Player'
player_columns = [col for col in test_data.columns if 'Player' in col]

# Keep only the columns with the word 'Player'
test_data = test_data[player_columns]
test_data=test_data.drop(columns=['Player_1','Player_2','Player_1_Odds','Player_2_Odds'])

In [9]:
import numpy as np

# Assuming your DataFrame is named 'test_data'
data['fav_percent_adj'] = np.ceil(data['fav_percent'] * 10) / 10
data['dog_percent_adj'] = np.ceil(data['dog_percent'] * 10) / 10



In [10]:
# Assuming your DataFrame is named 'test_data'
result = data.groupby(['fav_percent_adj', 'dog_percent_adj'])['Fav_Win'].agg(['mean', 'count']).reset_index()

result.rename(columns={'mean': 'Fav_Win_Percentage', 'count': 'Match_Count'}, inplace=True)


result.sort_values(by='Fav_Win_Percentage').to_pickle('Fav_Win_Percentage_Clay')

In [11]:
hard_today=data
Fav_Win_Percentage=result
hard_today['fav_percent_adj'] = np.ceil(hard_today['fav_percent'] * 10) / 10
hard_today['dog_percent_adj'] = np.ceil(hard_today['dog_percent'] * 10) / 10
hard_today_win_percent=pd.merge(hard_today,Fav_Win_Percentage,left_on=['fav_percent_adj','dog_percent_adj'],right_on=['fav_percent_adj','dog_percent_adj'])
#hard_today_win_percent=hard_today_win_percent[hard_today_win_percent['Resulted']!='Falxse']
hard_today_win_percent=hard_today_win_percent[hard_today_win_percent['Sex']!='Womens']
results=hard_today_win_percent[['Fav_Win_Percentage','Fav','Fav_Odds','fav_percent','dog_percent','Dog','Dog_Odds','Fav_Win']]#.sort_values(by=['Time'],ascending=True )

results=results[results['Fav_Win_Percentage']<.50]
len(results[results['Fav_Win']==True])/len(results),len(results)

(0.25316455696202533, 79)

In [12]:
data

,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,dog_rank,Elo_Fav_Elo,Elo_Dog_Elo,Fav_Top100,Dog_Top100,fav_rank_high,dog_rank_high,Fav_Win,fav_percent_adj,dog_percent_adj
0,2024-04-23,1.95,Tristan McCormick,1.78,1.95,Bogdan Bobrov,Tristan McCormick,10,Tristan McCormick,7,...,701.0,1458.774537,1454.641023,0.0,0.0,361.0,495.0,False,0.2,0.3
1,2024-04-23,1.72,Dennis Novak,1.72,2.06,Dennis Novak,Dennis Novak,13,Lukas Neumayer,12,...,206.0,1638.631043,1546.298759,0.0,0.7,85.0,206.0,True,0.4,0.3
5,2024-04-23,1.91,Arthur Gea,1.82,1.91,Kyle Edmund,Arthur Gea,8,Arthur Gea,10,...,404.0,1505.331089,1429.372402,0.0,0.0,14.0,394.0,False,0.3,0.2
7,2024-04-23,2.04,Henri Squire,1.73,2.04,Gabriel Diallo,Gabriel Diallo,6,Henri Squire,12,...,235.0,1583.342255,1533.866741,0.0,0.0,129.0,235.0,False,0.5,0.2
13,2024-04-23,1.77,Duje Ajdukovic,1.77,2.03,Duje Ajdukovic,Pierre Hugues Herbert,10,Pierre Hugues Herbert,10,...,143.0,1706.886990,1654.557430,0.0,0.0,115.0,36.0,True,0.3,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,2024-04-24,2.08,Zdenek Kolar,1.71,2.08,Jakub Nicod,Jakub Nicod,10,Zdenek Kolar,10,...,225.0,1670.957784,1542.208839,0.0,0.0,347.0,111.0,False,0.8,0.5
953,2024-04-24,1.79,Thiago Seyboth Wild,1.79,2.02,Thiago Seyboth Wild,Thiago Seyboth Wild,12,Roman Safiullin,11,...,42.0,1800.386154,1673.295906,0.4,0.5,63.0,36.0,True,0.5,0.6
954,2024-04-24,2.10,Donna Vekic,1.74,2.10,Laura Siegemund,Laura Siegemund,7,Donna Vekic,10,...,38.0,1623.056780,1568.422001,0.2,0.3,27.0,19.0,False,0.6,0.5
956,2024-04-24,1.82,Arthur Rinderknech,1.82,1.99,Arthur Rinderknech,Arthur Rinderknech,12,Alexander Shevchenko,11,...,59.0,1685.354283,1566.218169,0.4,0.3,42.0,45.0,True,0.5,0.7
